#L1 Regularized SVM and Logistic Regression using Stochastic Gradient Descent algorithm
###Author: Pablo Campos V.

##Necessary libraries

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.svm import l1_min_c
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from datetime import datetime
from sklearn.linear_model import SGDClassifier

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days
Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


##Let's create some fake data and scale the feature matrix

In [11]:
np.random.seed(123)
X = np.random.rand(1000,500)
y = np.random.randint(2, size=1000)
X_scaled = preprocessing.scale(X)

##Let's make a grid for the regularization parameter

In [4]:
cs = np.linspace(0.0001, 1, num=100)

##L1 Regularized Linear SVM: Compute the trajectory of variables using SGDClassifier

In [16]:
start = datetime.now()
clf = SGDClassifier(loss='hinge', penalty='l1',alpha=0.001, n_iter=200, fit_intercept=True, l1_ratio=1)
coefs_ = []
for c in cs:
    clf.set_params(alpha=c)
    clf.fit(X_scaled, y)
    coefs_.append(clf.coef_.ravel().copy())
print("Time ", datetime.now() - start)

('Time ', datetime.timedelta(0, 94, 618283))


##Visualizing the trajectory

In [18]:
coefs_ = np.array(coefs_)
plt.plot(np.log10(cs), coefs_)
ymin, ymax = plt.ylim()
plt.xlabel('log(C)')
plt.ylabel('Coefficients')
plt.title('L1 Regularized SVM Trajectory')
plt.axis('tight')
plt.show()

##L1 Regularized Logistic Regression: Compute the trajectory of variables using SGDClassifier

In [10]:
start = datetime.now()
clf = SGDClassifier(loss='log', penalty='l1',alpha=0.01, n_iter=200, fit_intercept=True, l1_ratio=1,
                    random_state=123)
aucs = np.zeros(len(cs))
coefs_ = []
i=0
for c in cs:
    clf.set_params(alpha=c)
    clf.fit(X_scaled, y)
    coefs_.append(clf.coef_.ravel().copy())
    prob = clf.predict_proba(X_scaled)
    prob = prob[:,1]
    aucs[i]=roc_auc_score(y, prob)
    i = i+1
print("Time ", datetime.now() - start)


('Time ', datetime.timedelta(0, 98, 72530))


##Visualizing the trajectory

In [17]:
coefs_ = np.array(coefs_)
plt.plot(np.log10(cs), coefs_)
ymin, ymax = plt.ylim()
plt.xlabel('log(C)')
plt.ylabel('Coefficients')
plt.title('L1 Regularized Logistic Regression Trajectory')
plt.axis('tight')
plt.show()

##We can store the coefficients matrix in SciPy sparse format (memory efficient)

In [66]:
sp_coefs_ = sp.sparse.csr_matrix(coefs_)

##Computing area under ROC curve on train data as function of regularization parameter

In [37]:
plt.plot(np.log10(cs), aucs)
ymin, ymax = plt.ylim()
plt.xlabel('log(C)')
plt.ylabel('AUC')
plt.title('Regularization parameter vs AUC')
plt.axis('tight')
plt.show()

##Let's create some test data

In [12]:
np.random.seed(456)
X_test = np.random.rand(1000,500)
y_test = np.random.randint(2, size=1000)

##Scale the test data according to train data statistics

In [13]:
scaler = preprocessing.StandardScaler().fit(X)
X_test_scaled = scaler.transform(X_test) 

##L1 Regularized Logistic Regression: Compute the trajectory of variables using SGDClassifier
###Predictions on test data

In [14]:
start = datetime.now()
clf = SGDClassifier(loss='log', penalty='l1',alpha=0.01, n_iter=200, fit_intercept=True, l1_ratio=1,
                    random_state=123)
aucs = np.zeros(len(cs))
coefs_ = []
i=0
for c in cs:
    clf.set_params(alpha=c)
    clf.fit(X_scaled, y)
    coefs_.append(clf.coef_.ravel().copy())
    prob = clf.predict_proba(X_test_scaled)
    prob = prob[:,1]
    aucs[i]=roc_auc_score(y_test, prob)
    i = i+1
print("Time ", datetime.now() - start)

('Time ', datetime.timedelta(0, 98, 506011))


##Computing area under ROC curve on test data as function of regularization parameter

In [15]:
plt.plot(np.log10(cs), aucs)
ymin, ymax = plt.ylim()
plt.xlabel('log(C)')
plt.ylabel('AUC')
plt.title('Regularization parameter vs AUC')
plt.axis('tight')
plt.show()